In [1]:
import numpy as np
import matplotlib.pyplot as plt
from pygsp import graphs, filters, plotting
import pygsp as pg
from numpy.random import default_rng
from numpy.random import randint
from random import sample
import random
import cvxpy as cp

plotting.BACKEND = 'matplotlib'
plt.rcParams['figure.figsize'] = (10, 5)

In [2]:
def createGraph(Nodes,i):
    graphType = randint(0, 6, 1)
    while True:
        s = np.array(sample(range(1, 10000), 1))
        if graphType==0:
            G = pg.graphs.BarabasiAlbert(N = Nodes,seed = s)
            typeName = 'BarabasiAlbert'
        elif graphType == 1:
            G = pg.graphs.RandomRegular(N = Nodes,seed = s)
            typeName = 'RandomRegular'
        elif graphType == 2:
            G = pg.graphs.DavidSensorNet(N = Nodes,seed = s)
            typeName = 'DavidSensorNet'
        elif graphType == 3:
            G = pg.graphs.Community(N = Nodes,Nc = 3,seed = s)
            typeName = 'Community'
        elif graphType == 4:
            G = pg.graphs.ErdosRenyi(N = Nodes,p = 0.1,seed = s)
            typeName = 'ErdosRenyi'
        else:
            G = pg.graphs.Sensor(N=Nodes,seed = s)
            typeName = 'Sensor'
            
        if (G.is_connected() and not(G.is_directed())):
            break
            
    print(typeName)
           
    G.compute_fourier_basis()
    W = np.random.uniform(size=(2,1))
    x =  G.U[:,1:3]@W
    x = x/np.max(np.abs(x))
    eigenvalue = G.e
    print(eigenvalue[1:4])

    return G,x

In [3]:
def computeMainSignal(signals,k,N):
    x = np.zeros((N,1))
    for i in range(k):
        x = x + signals[:,i].reshape(N,1)
    return x

In [4]:
def computeSmoothness(signals,graphs,k):
    smoothness = []
    sumOverSmoothness = 0
    for i in range(k):
        smoothness.append(signals[:,i].T@graphs[i].L@signals[:,i])
        sumOverSmoothness = sumOverSmoothness + smoothness[i]
    return sumOverSmoothness,smoothness

In [5]:
def decomposeGraphSignal(mainSignal,signals,graphs,k,N):    
    # Create A , H
    w = np.arange(1,40*(N-1),40)
    H = np.diag(np.tile(w, (1, k))[0])
    for i in range(k):
        G = graphs[i]
        A_init = G.U[:, 1:]
        if i==0:
            A = A_init
        else:
            A = np.hstack((A, A_init))

    # solve by CVX
    Y = cp.Variable(((N-1)*k,1))
    objective = cp.Minimize(cp.quad_form(Y, H))  
    # equality 
    constraints = [A @ Y == mainSignal]
    prob = cp.Problem(objective, constraints)
    result = prob.solve(verbose=True)
    Y_hat = np.vstack((np.zeros((1,k)),Y.value.reshape(N-1,k,order='F')))

    # IGFT
    X_hat = np.zeros((N,k))
    for i in range(k):
        G = graphs[i]
        X_hat[:,i] = (G.U @ Y_hat[:,i])
    
    return X_hat

In [6]:
def calculateSNR(est_signals,signals,k):
    SNR = np.zeros((1,k))
    for i in range(k):
        SNR[0,i]=(10*np.log10(np.linalg.norm(signals[:,i])**2/np.linalg.norm(signals[:,i]-est_signals[:,i])**2))
    return SNR

In [7]:
N = 450   # number of nodes
k = 5     # number of graphs
np.random.seed(0)
random.seed(0)
iteration = 200
av_snr = np.zeros((1,k))

for itera in range(1,iteration+1):
    print('############################################# experiment = ',itera,' ########################################')
    # create graph and graph signals
    graph_list = []
    signals = np.zeros((N,k))
    for i in range(k):
        G , x = createGraph(N,i)
        graph_list.append(G)
        signals[:,[i]] = x

    # calculate the main signal : x = x1+x2+...+x4
    mainSignal = computeMainSignal(signals,k,N)
    mainSignal = mainSignal - np.mean(mainSignal)

    # decompose graph signal
    estimated_signals = decomposeGraphSignal(mainSignal,signals,graph_list,k,N)

    # calculate graph signals' smoothness
    sumOverSmoothness,smoothness = computeSmoothness(signals,graph_list,k)
    print('smoothness:')
    print(smoothness)
    print('sum over smoothness:',sumOverSmoothness)

    # calculate estimated graph signals' smoothness
    sumOverSmoothness_est,smoothness_est = computeSmoothness(estimated_signals,graph_list,k)
    print('smoothness_est:')
    print(smoothness_est)
    print('sum over smoothness_est:',sumOverSmoothness_est)

    # calculate SNRs
    SNRs = calculateSNR(estimated_signals,signals,k)
    print('SNR:',SNRs)
    av_snr = av_snr + SNRs/iteration

    # compute objective values
    objectiveValue = sumOverSmoothness
    objectiveValue_est = sumOverSmoothness_est
    print('objectiveValue:',objectiveValue)
    print('objectiveValue_est:',objectiveValue_est)

    # compute error
    error = np.linalg.norm(mainSignal-estimated_signals@np.ones((k,1)))
    print('error:',error)
    
    
print('av_snr: ',av_snr)
print('total_av_snr: ',np.mean(av_snr))

############################################# experiment =  1  ########################################


2021-12-03 15:10:20,762:[INFO](pygsp.graphs.community.__init__): Constructed using eps-NN with eps = 3.2567778121631528


ErdosRenyi
[26.88731118 27.59783794 28.33788505]


2021-12-03 15:10:21,554:[INFO](pygsp.graphs.community.__init__): Constructed using eps-NN with eps = 3.2567778121631528
2021-12-03 15:10:22,426:[INFO](pygsp.graphs.community.__init__): Constructed using eps-NN with eps = 3.2567778121631528
2021-12-03 15:10:23,114:[INFO](pygsp.graphs.community.__init__): Constructed using eps-NN with eps = 3.2567778121631528
2021-12-03 15:10:23,676:[INFO](pygsp.graphs.community.__init__): Constructed using eps-NN with eps = 3.2567778121631528
2021-12-03 15:10:24,198:[INFO](pygsp.graphs.community.__init__): Constructed using eps-NN with eps = 3.2567778121631528


Community
[0.08673535 0.31674044 0.53815972]
Sensor
[0.06809214 0.07604609 0.13376286]
BarabasiAlbert
[0.00658864 0.0088679  0.01365717]
BarabasiAlbert
[0.00358498 0.00655854 0.00893554]
-----------------------------------------------------------------
           OSQP v0.6.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2019
-----------------------------------------------------------------
problem:  variables n = 2245, constraints m = 450
          nnz(P) + nnz(A) = 1011976
settings: linear system solver = qdldl,
          eps_abs = 1.0e-05, eps_rel = 1.0e-05,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 10000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: on, time_limit: off

iter   objective    pri res    dua re

...

Community
[0.40835554 0.54063676 0.5548798 ]


2021-12-03 15:45:36,342:[INFO](pygsp.graphs.community.__init__): Constructed using eps-NN with eps = 3.2567778121631528


BarabasiAlbert
[0.00552521 0.0097957  0.01021744]


2021-12-03 15:45:36,746:[INFO](pygsp.graphs.community.__init__): Constructed using eps-NN with eps = 3.2567778121631528


Community
[0.15878746 0.34766957 0.4053732 ]
-----------------------------------------------------------------
           OSQP v0.6.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2019
-----------------------------------------------------------------
problem:  variables n = 2245, constraints m = 450
          nnz(P) + nnz(A) = 1012186
settings: linear system solver = qdldl,
          eps_abs = 1.0e-05, eps_rel = 1.0e-05,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 10000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: on, time_limit: off

iter   objective    pri res    dua res    rho        time
   1   0.0000e+00   1.98e+00   1.71e+06   1.00e-01   8.52e-01s
  50   6.2100e+03   3.31e-09   2.31e-05   1.00e-01   1.08e